In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [32]:
dat = pd.read_csv('../data/processed/rain_X_train.csv', header=None, nrows=200000)

In [33]:
dat.rename({0: 'oid'}, axis=1, inplace=True)

In [47]:
cts = pd.DataFrame(dat.groupby('oid')['oid'].value_counts().droplevel(0))
cts.columns = ['counts']
cts

,counts
oid,
1.0,64
2.0,27
3.0,2
4.0,5
5.0,5
...,...
14963.0,4
14964.0,7
14965.0,4


In [35]:
seq_lengths = cts.reset_index().groupby('counts')['counts'].value_counts().droplevel(0)
seq_lengths

counts
1      2248
2      2628
3      1595
4      1070
5       831
       ... 
434       1
457       1
477       1
528       1
621       1
Name: counts, Length: 236, dtype: int64

In [43]:
sizes = np.array(seq_lengths.index)

# randomize the order of the sizes for batching
np.random.choice(sizes, sizes.shape[0], replace=False)

array([ 67,  58,   4, 119, 105,  46,  88,  94, 213, 139,  52,  78, 302,
        75,  63, 106, 134, 254,  39, 149, 114, 150, 133, 200, 192,  64,
        73, 120, 108, 160, 218,  44, 308, 132,  90, 102,  59, 337,  45,
       199, 182,  96,  37,  11,   6, 170,  51, 398,  60, 208, 162, 359,
        69, 301, 299, 100,  25, 147,  32,  49, 121,  28, 231, 169,  43,
       187, 174, 128, 277,  83, 193,  13,  68, 117,  24, 125, 113,  71,
       477,  65,  91,  86, 123,  10,  22, 138,  19, 118,  82, 184, 411,
        54,   5, 122, 161,  30,  42,  95, 137, 249, 156, 351,  84, 201,
       157, 159, 141, 214, 181, 171, 311, 176,  66, 224,  98,  34, 144,
        89, 221, 228, 434,  17, 309,  92, 426,  20, 115, 146, 198,  72,
       272, 130, 255, 143, 188, 154, 333,  74,   3, 126,  40, 261, 318,
        48, 183,  93, 145, 268,  23,  14, 264, 240,  81, 322, 262, 194,
       457, 312, 135,  18,  99, 166, 101, 232, 210, 136, 152, 175,  57,
        29,  97, 104, 190,  56, 151, 621,  12,  80,  70,  21,  3

In [61]:
# use for batching with the same size
oids_by_size_dict = {
    size: cts.query('counts == @size').index.tolist() for size in sizes
}
oids_by_size_dict

{1: [2732.0,
  2734.0,
  2735.0,
  2738.0,
  2745.0,
  2754.0,
  2757.0,
  2765.0,
  2766.0,
  2769.0,
  2770.0,
  2772.0,
  2773.0,
  2774.0,
  2781.0,
  2796.0,
  2799.0,
  2809.0,
  2815.0,
  2816.0,
  2818.0,
  2826.0,
  2829.0,
  2833.0,
  2837.0,
  2844.0,
  2845.0,
  2847.0,
  2849.0,
  2852.0,
  2853.0,
  2854.0,
  2859.0,
  2860.0,
  2861.0,
  2865.0,
  2866.0,
  2867.0,
  2873.0,
  2890.0,
  2892.0,
  2894.0,
  2895.0,
  2902.0,
  2904.0,
  2908.0,
  2913.0,
  2916.0,
  2917.0,
  2940.0,
  2947.0,
  2949.0,
  2955.0,
  2969.0,
  2974.0,
  2981.0,
  2989.0,
  2994.0,
  2995.0,
  3000.0,
  3008.0,
  3015.0,
  3022.0,
  3024.0,
  3026.0,
  3027.0,
  3042.0,
  3047.0,
  3066.0,
  3068.0,
  3080.0,
  3085.0,
  3088.0,
  3091.0,
  3097.0,
  3111.0,
  3117.0,
  3119.0,
  3131.0,
  3138.0,
  3147.0,
  3150.0,
  3164.0,
  3166.0,
  3176.0,
  3179.0,
  3189.0,
  3191.0,
  3193.0,
  3195.0,
  3207.0,
  3208.0,
  3209.0,
  3214.0,
  3223.0,
  3224.0,
  3225.0,
  3235.0,
  3236.0,
  3245.

In [ ]:
def train_epoch(X, y, max_batch_size=32):
    
    # randomize sizes
    
    # for size in rand_sizes...
    
    # get complete tensor where X.shape is (n_obs @ size, time_size, n_features) and y.shape is similar
    
    # get list of oids and randomize
    
    # organize oids into a list of lists where each list has a length of max_batch_size (except the last one obviously)
    
    # generate batches where batch_X.shape is (<= max_batch_size, time_size, n_features)

    # drop the oid from batch_X and batch_y
    
    